In [1]:
import os
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

addedToTrainingData = False
DATA_COLUMNS = ["A","B","C","D","E","F","G","DIGIT"]
leds = np.array([[1,1,1,1,1,1,0],
                     [0,1,1,0,0,0,0],
                     [1,1,0,1,1,0,1],
                     [1,1,1,1,0,0,1],
                     [0,0,1,0,0,1,1],
                     [1,0,1,1,0,1,1],
                     [1,0,1,1,1,1,1],
                     [1,1,1,0,0,0,0],
                     [1,1,1,1,1,1,1],
                     [1,1,1,1,0,1,1]
                    ])

input_data = []
for indx in range(len(leds)):
    led = [np.append(leds[indx], indx)]
    if len(input_data) == 0:
        input_data = led
    else:
        input_data = np.concatenate((input_data, led), axis=0)

In [2]:
leds_strength = np.sum(leds, axis=1)

In [3]:
leds_importance = np.divide(leds, leds_strength[:,None])
total_leds_importance = np.sum(leds_importance, axis=0)

In [4]:
leds_relative_importance = np.divide(leds_importance, total_leds_importance)

In [5]:
def find_2_least_important_segments(l):
    leds = l.copy()
    leds[leds == 0] = 2
    least_important_segments = []
    for indx in range(len(leds)):
        led = leds[indx]
        seg1 = led.argmin()
        led[seg1] = 2
        seg2 = led.argmin()
        least_important_segments.append((seg1, seg2))
        
    return least_important_segments 

In [6]:
least_imp_segs = find_2_least_important_segments(leds_relative_importance)
print("Least imp segments", least_imp_segs)

Least imp segments [(2, 1), (2, 1), (1, 0), (2, 1), (2, 6), (2, 0), (2, 0), (2, 1), (2, 1), (2, 1)]


In [7]:
def stringify_array(arr):
    arr = arr.flatten()
    array_str = None
    for val in arr:
        if array_str == None:
            array_str = str(val)
        else:
            array_str = array_str + "," + str(val)
    
    return array_str

In [8]:
dict = {}
for indx in range(len(leds)):
    (seg1, seg2) = least_imp_segs[indx]
    led = leds[indx]
    v = led[seg1]
    led[seg1] = 0
    led_str = stringify_array(led)
    if dict.__contains__(led_str):
        print("Clash1", led_str, dict.get(led_str), indx, seg1)
        if leds_relative_importance[dict.get(led_str)][seg1] < leds_relative_importance[indx][seg1]:
            print("Updating dict")
            dict[led_str] = indx
    else:
        dict[led_str] = indx
    led[seg1] = v
    
    v = led[seg2]
    led[seg2] = 0
    led_str = stringify_array(led)
    if dict.__contains__(led_str):
        print("Clash2", led_str, dict.get(led_str), indx, seg2)
        if leds_relative_importance[dict.get(led_str)][seg2] < leds_relative_importance[indx][seg2]:
            print("Updating dict")
            dict[led_str] = indx
    else:
        dict[led_str] = indx
    led[seg2] = v
    
    v1 = led[seg1]
    v2 = led[seg2]
    led[seg1] = 0
    led[seg2] = 0
    led_str = stringify_array(led)
    
    if dict.__contains__(led_str):
        print("Clash", led_str, dict.get(led_str), indx, seg1, seg2)
        if leds_strength[dict.get(led_str)] > leds_strength[indx]:
            print("Updating dict")
            dict[led_str] = indx
    else:
        dict[led_str] = indx
        
    led[seg1] = v1
    led[seg2] = v2


Clash 1,0,0,1,1,1,1 6 8 2 1
Clash 1,0,0,1,0,1,1 5 9 2 1


In [9]:
validation_data = []
for led_str in dict.keys():
    led = np.fromstring(led_str, dtype=int, sep=',')
    led = [np.append(led, dict.get(led_str))]
    
    if len(validation_data) == 0:
        validation_data = led
    else:
        validation_data = np.concatenate((validation_data, led), axis=0)
    
    if addedToTrainingData:
        input_data = np.concatenate((input_data, led), axis=0)

if not addedToTrainingData:
    each = input_data.copy()
    input_data = np.concatenate((input_data, each), axis=0)
    input_data = np.concatenate((input_data, each), axis=0)

In [10]:
train_data = pd.DataFrame(input_data, columns=DATA_COLUMNS,)

TypeError: __init__() got an unexpected keyword argument 'index_col'

In [ ]:
test_data = pd.DataFrame(validation_data, columns=DATA_COLUMNS)

In [ ]:
train_data.head()

In [ ]:
test_data.head()

In [ ]:
def find_all_cases(arr):
    if len(arr) == 7:
        return [arr]
    
    return np.concatenate((find_all_cases(np.append(arr, 0)),
    find_all_cases(np.append(arr, 1))), axis=0)
    
seed_arr = []
all_cases_input_data = pd.DataFrame(np.int32(find_all_cases(seed_arr)), columns = DATA_COLUMNS[0 : len(DATA_COLUMNS) - 1])


In [ ]:
all_cases_input_data.head()

In [ ]:
all_cases_input_data.to_csv("all_cases_input_data.csv", sep=',', index_label=False)

In [ ]:
train_data.to_csv("train_data.csv", sep=',', index_label=False)

In [ ]:
test_data.to_csv("test_data.csv", sep=',', index_label=False)

In [ ]:
train_data.head()